In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

p=0

# params=np.random.uniform(0,1,23)

proj = np.array([[1, 0], [0, 0]])

# Selecting the device
dev = qml.device("default.mixed", wires=3)

# Function to create the CHSH observables
def create_observable(theta, phi):
    return np.array([[np.cos(theta), np.exp(-1j * phi) * np.sin(theta)],
                     [np.exp(1j * phi) * np.sin(theta), -np.cos(theta)]])


# Bell pair circuit
def bell_pair(params):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.U3(params[0],params[1],params[2],wires=1)
    qml.U3(params[3],params[4],params[5],wires=2)
    qml.CNOT(wires=[2,1])
    qml.RZ(params[6],wires=1)
    qml.RY(params[7],wires=2)
    qml.CNOT(wires=[1,2])
    qml.RY(params[8],wires=2)
    qml.CNOT(wires=[2,1])
    qml.U3(params[9],params[10],params[11],wires=1)
    qml.U3(params[12],params[13],params[14],wires=2)
    qml.PhaseFlip(1-p,wires=1)
    qml.PhaseFlip(1-p,wires=2)
    qml.adjoint(qml.U3(params[12],params[13],params[14],wires=2))
    qml.adjoint(qml.U3(params[9],params[10],params[11],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.RY(params[8],wires=2))
    qml.CNOT(wires=[1,2])
    qml.adjoint(qml.RY(params[7],wires=2))
    qml.adjoint(qml.RZ(params[6],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.U3(params[3],params[4],params[5],wires=2))
    qml.adjoint(qml.U3(params[0],params[1],params[2],wires=1))
    # post selection
    qml.Hermitian(proj, wires=2)

# Measure AB function
def measure_AB(dev, bell_pair, A, B):
    @qml.qnode(dev)
    def circuit(params):
        bell_pair(params[:15])  # First 15 parameters for Bell pair
        return qml.expval(qml.Hermitian(A, wires=0) @ qml.Hermitian(B, wires=1))
    return circuit

# Cost function
def cost(params):
    # Creating observables with new parameters
    A1 = create_observable(params[15], params[16])
    A2 = create_observable(params[17], params[18])
    B1 = create_observable(params[19], params[20])
    B2 = create_observable(params[21], params[22])
    

    expvals = [measure_AB(dev, bell_pair, A1, B1)(params), 
               measure_AB(dev, bell_pair, A1, B2)(params),
               measure_AB(dev, bell_pair, A2, B1)(params),
               measure_AB(dev, bell_pair, A2, B2)(params)]

    return -(np.sum(expvals[:3]) - expvals[3])

# Optimizer 1
# opt = qml.GradientDescentOptimizer(stepsize=0.1)

# # Initialize parameters: 15 for Bell pair + 8 for CHSH observables
# params = [0, 1.96948032,  2.54700987,  1.68446586,  2.79762231, -0.33795471,
#  -0.81783546, -1.75707826, -2.94140176, -0.90088855,  0.18664063, -0.31905494,
#   1.90106382, -2.04491857, -2.43084329,0,0,0.78,0,0.39,0,1.2,0]

# # Optimization loop
# for i in range(100):
#     params, ccost = opt.step_and_cost(cost, params)
#     print(f"cost={ccost}, parameters={params}")

# Using Adam optimizer
#opt = qml.AdamOptimizer(stepsize=0.01)
opt = qml.SPSAOptimizer(maxiter=1, alpha=0.702, gamma=0.101, c=0.2, A=1, a=0.2)
# Initialize parameters as a PennyLane tensor, ensuring they are recognized as trainable
np.random.seed(42)
params = np.array(np.random.uniform(-0.9, 0.9, 23), requires_grad=True)

# params[15]=0 
# params[16]=0
# params[17]=0.78
# params[18]=0
# params[19]=0.39
# params[20]=0
# params[21]=-0.39
# params[22]=0
# Optimization loop with increased iterations and detailed logging
for i in range(1001):  # Increased iterations
    params, ccost = opt.step_and_cost(cost, params)
    if i % 100 == 0:  # Detailed logging every 10 iterations
        print(f"Iteration {i}: cost = {-ccost}, parameters = {params}")

# You can add more code here to analyze the final parameters or visualize the optimization process



Iteration 0: cost = 1.5122271728349401, parameters = [-0.378022    0.96347996  0.26539489  0.02539106 -0.46697224 -0.77140407
 -0.64325529  0.50692285  0.33420123  0.22233643 -1.01514212  0.99803194
  0.75059096 -0.36559539 -0.42052085 -0.41767767 -0.20016975  0.19675579
  0.02969524 -0.52798176  0.049141   -0.49671684 -0.52633384]
Iteration 100: cost = 2.0640418071082505, parameters = [-1.97963640e-04  1.27117890e+00 -8.18402578e-02  1.62191558e-01
 -6.94821709e-01 -1.64881303e+00 -4.84283931e-01  1.40890003e+00
  6.04179811e-01  7.43308742e-02 -5.72416127e-01  1.49413360e-01
  4.76419723e-01 -9.85936127e-01 -6.00721498e-02  6.93636001e-02
 -1.53121021e-01 -1.68108077e-01 -7.74154969e-02 -8.80484572e-02
 -5.19345027e-01 -7.17014064e-01 -3.29468906e-01]
Iteration 200: cost = 2.2617948709638998, parameters = [-0.05730038  1.27332794 -0.12004148  0.14982867 -0.71571208 -1.65097948
 -0.48002717  1.45129844  0.59639748  0.0613964  -0.56460106  0.14644882
  0.29387446 -0.99366652 -0.0849231